In [15]:
import nltk
import numpy as np

nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu

[nltk_data] Downloading package punkt to /home/grad3/hle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [88]:
references = np.load('references_from_0_to_1000.npy', allow_pickle=True)
expressions_1 = np.load('top3_exps_from_0_to_1000.npy', allow_pickle=True)

In [89]:

tokenized_ref = []
ref_1 = [r[0] for r in references[1]]
print(ref_1)
for r in ref_1:
    lis=word_tokenize(r.lower())
    tokenized_ref.append(lis)
print(tokenized_ref)
print(expressions_1[1])

['cat', 'Cat on right', 'cat, but not in reflection']
[['cat'], ['cat', 'on', 'right'], ['cat', ',', 'but', 'not', 'in', 'reflection']]
['carpet' 'cat' 'dog']


In [96]:
BLEUscore = nltk.translate.bleu_score.sentence_bleu(tokenized_ref, word_tokenize(expressions_1[1][1]))
print("BLEU:",BLEUscore)

BLEU: 1.0


/scratch3/hle/conda_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [91]:
bleu_score = []
counter = 0
files = [0,1000,2000,3000,4000]
for start in files:
    references = np.load(f'references_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    expressions = np.load(f'top3_exps_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    for ref, exps in zip(references, expressions):
        item_reference = [word_tokenize(r[0].lower()) for r in ref]
    #     print(item_reference)
    #     print(exps)
    #     print('$$$$$$$$$$$')

        best_matched = max([sentence_bleu(item_reference, word_tokenize(exp)) for exp in exps])
        bleu_score.append(best_matched)
print(len(bleu_score))

/scratch3/hle/conda_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/scratch3/hle/conda_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


5000


In [102]:
x = [score for score in bleu_score if score != 0]
print(np.average(x), np.std(x))
print(np.average(bleu_score), np.std(bleu_score))
print(len(x))
# 0.3939225254870882 0.3270738202276801
# 0.1983005993302002 0.3043752780752566

0.38507907917574985 0.3187066035683757
0.24899213259503986 0.3155348827674987
3233


### CALCULATE ROUGE SCORE


In [39]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [98]:
references = np.load('references_from_0_to_1000.npy', allow_pickle=True)
expressions_1 = np.load('top3_exps_from_0_to_1000.npy', allow_pickle=True)

In [99]:
cat_ref = references[1]
cat_exps = expressions_1[1]
print(cat_ref, cat_exps)

[['cat'], ['Cat on right'], ['cat, but not in reflection']] ['carpet' 'cat' 'dog']


In [100]:
test_scores = []
for i in cat_ref:
    score = scorer.score(i[0], cat_exps[1])
    print(score['rouge1'], score['rougeL'])
    test_scores.append(score['rouge1'])
print(np.average(test_scores))

Score(precision=1.0, recall=1.0, fmeasure=1.0) Score(precision=1.0, recall=1.0, fmeasure=1.0)
Score(precision=1.0, recall=0.3333333333333333, fmeasure=0.5) Score(precision=1.0, recall=0.3333333333333333, fmeasure=0.5)
Score(precision=1.0, recall=0.2, fmeasure=0.33333333333333337) Score(precision=1.0, recall=0.2, fmeasure=0.33333333333333337)
0.7074074074074074


In [101]:
rouge1_scores = []
rougeL_scores = []
for start in files:
    references = np.load(f'references_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    expressions = np.load(f'top3_exps_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    for ref, exps in zip(references, expressions):
        item_reference = [word_tokenize(r[0].lower()) for r in ref]
        exp_recalls_rouge_1 = []
        exp_recalls_rouge_L = []
        for exp in exps:
            exp_recalls_rouge_1.append(np.average([scorer.score(target[0], exp)['rouge1'].recall for target in ref]))
            exp_recalls_rouge_L.append(np.average([scorer.score(target[0], exp)['rougeL'].recall for target in ref]))
        rouge1_scores.append(max(exp_recalls_rouge_1))
        rougeL_scores.append(max(exp_recalls_rouge_L))
print(len(rouge1_scores))

5000


In [103]:
nonzero_rouge1 = [score for score in rouge1_scores if score != 0]
print(np.average(nonzero_rouge1), np.std(nonzero_rouge1))
print(np.average(rouge1_scores), np.std(rouge1_scores))

0.24809894991750528 0.16986040769537178
0.1687072859439036 0.18169670240599997


In [104]:
nonzero_rougeL = [score for score in rougeL_scores if score != 0]
print(np.average(nonzero_rougeL), np.std(nonzero_rougeL))
print(np.average(rougeL_scores), np.std(rougeL_scores))

0.2480695381527994 0.1698341010559976
0.1686872859439036 0.18167124031241783


In [70]:


references = np.load('references_from_0_to_1000.npy', allow_pickle=True)
expressions_1 = np.load('top3_exps_from_0_to_1000.npy', allow_pickle=True)

#### Add meteor metric (include synonims)
#### Improve matching algorithm (prioritizing matched objects with synonims)
#### Add relations to RSA ?
#### Generate attributes/objects/relations from scenegraph on the training dataset.
#### Learn threshold (entropy & for relations?) from the training data
#### Idea: treating nodes as objects & edges as relations. -> generate expressions.

In [106]:
# FINDING ALL IMAGES WITH 0 SCORE EXPRESSION
bad_bleu_score_images = []
for i in range(len(bleu_score)):
    if bleu_score[i] == 0:
        bad_bleu_score_images.append(i)
print(len(bad_bleu_score_images))
print(bad_bleu_score_images[:10])

bad_rouge1_score_images = []
bad_rougeL_score_images = []
for i in range(len(rouge1_scores)):
    if rouge1_scores[i] == 0:
        bad_rouge1_score_images.append(i)
    if rougeL_scores[i] == 0:
        bad_rougeL_score_images.append(i)
print(len(bad_rouge1_score_images))
print(bad_rouge1_score_images[:10])
print(len(bad_rougeL_score_images))
print(bad_rougeL_score_images[:10])

1767
[0, 2, 4, 9, 12, 14, 20, 21, 24, 27]
1600
[0, 9, 12, 14, 21, 24, 27, 30, 35, 36]
1600
[0, 9, 12, 14, 21, 24, 27, 30, 35, 36]


In [107]:
rouge_bad_intersection = np.intersect1d(bad_rougeL_score_images, bad_bleu_score_images)
print(len(rouge_bad_intersection))
bleu_rouge_bad_intersection = np.intersect1d(bad_rougeL_score_images, bad_bleu_score_images)
print(len(bleu_rouge_bad_intersection))
bleu_rouge_bad_union = np.union1d(bad_rougeL_score_images, bad_bleu_score_images)
print(len(bleu_rouge_bad_union))

1600
1600
1767


In [87]:
np.save('bad_rouge_score_reference_indexes.npy',bad_rouge1_score_images)
np.save('bad_bleu_score_reference_indexes.npy',bad_bleu_score_images)